In [1]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import AksCompute
from azureml.core.workspace import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace, Model, Datastore, Dataset

subscription_id = 'YOUR_SUBSCRIPTION'
resource_group = 'YOUR_RESOURCE_GROUP'
workspace_name = 'YOUR_AML_WORKSPACE'

auth = InteractiveLoginAuthentication()

ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,auth= auth)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

datastore = ws.get_default_datastore()


Workspace name: YOUR_AML_WORKSPACE
Azure region: eastus
Subscription id: YOUR_SUBSCRIPTION
Resource group: YOUR_RESOURCE_GROUP


Installing Python 3.10 on Client

```
# compile python from source - avoid unsupported library problems
RUN apt update -y && sudo apt upgrade -y && \
    apt-get install -y wget build-essential checkinstall  libreadline-gplv2-dev  libncursesw5-dev  libssl-dev  libsqlite3-dev tk-dev libgdbm-dev libc6-dev libbz2-dev libffi-dev zlib1g-dev && \
    cd /usr/src && \
    sudo wget https://www.python.org/ftp/python/3.8.10/Python-3.8.10.tgz && \
    sudo tar xzf Python-3.8.10.tgz && \
    cd Python-3.8.10 && \
    sudo ./configure --enable-optimizations && \
    sudo make altinstall
```

In [2]:
### to update existing Webservice run this code here:

 #Ray Cluster deployment for UAT AKS Cluster

from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute
from azureml.core.model import Model

from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
import sklearn


environment = Environment('supplychainray310')
environment.environment_variables = {'SCORING_TIMEOUT_MS':6000000}


dockerfile = r'''

FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04
USER root:root

# apt-get and system utilities
RUN apt-get update && apt-get install -y \
   curl apt-utils apt-transport-https debconf-utils gcc build-essential g++\
   && rm -rf /var/lib/apt/lists/*
# adding custom MS repository
RUN curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
RUN curl https://packages.microsoft.com/config/ubuntu/18.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
# install libssl - required for sqlcmd to work on Ubuntu 18.04
RUN apt-get update && apt-get install -y libssl1.1 libssl-dev
# install SQL Server drivers
RUN apt-get update && ACCEPT_EULA=Y apt-get install -y msodbcsql17 unixodbc-dev
# install SQL Server tools
RUN apt-get update && ACCEPT_EULA=Y apt-get install -y mssql-tools
RUN echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
RUN /bin/bash -c "source ~/.bashrc"
## 2/3/23 - GG installing python 3.10 

# install necessary locales
RUN apt-get update && apt-get install -y locales \
   && echo "en_US.UTF-8 UTF-8" > /etc/locale.gen
RUN pip install --upgrade pip
'''


# Set the base image to None, because the image is defined by Dockerfile.
environment.docker.base_dockerfile = dockerfile
environment.docker.base_image = None
environment.inferencing_stack_version='latest'

environment.environment_variables = {'SCORING_TIMEOUT_MS':6000000}

conda_dep = CondaDependencies()

# Installs packages now
conda_dep.add_conda_package("pip")
conda_dep.add_pip_package("orjson")
conda_dep.add_pip_package("ray==2.2.0")
conda_dep.add_conda_package("numpy==1.23.5")
conda_dep.add_pip_package('azure-ai-ml==1.2.0')
conda_dep.add_pip_package('pyomo==5.7.3')
conda_dep.add_pip_package('cplex==22.1.0.0')
conda_dep.add_pip_package("pyodbc==4.0.35")
conda_dep.add_pip_package("joblib==1.2.0")
conda_dep.add_pip_package("sqlalchemy==1.4.44")
conda_dep.add_pip_package("protobuf==4.21.11")
conda_dep.add_pip_package("gluonts==0.11.4")
conda_dep.add_pip_package("mxnet==1.9.1")




# Adds dependencies to PythonSection of myenv
environment.python.conda_dependencies=conda_dep

environment.register(ws)


Property environment_variables is deprecated. Use RunConfiguration.environment_variables to set runtime variables.
Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None
Property environment_variables is deprecated. Use RunConfiguration.environment_variables to set runtime variables.


{
    "assetId": "azureml://locations/eastus/workspaces/5b0473ef-8a7f-412e-a5fc-1742cf0c4541/environments/supplychainray377/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "\n\nFROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04\nUSER root:root\n\n# apt-get and system utilities\nRUN apt-get update && apt-get install -y \\\n   curl apt-utils apt-transport-https debconf-utils gcc build-essential g++\\\n   && rm -rf /var/lib/apt/lists/*\n# adding custom MS repository\nRUN curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -\nRUN curl https://packages.microsoft.com/config/ubuntu/18.04/prod.list > /etc/apt/sources.list.d/mssql-release.list\n# install libssl - required for sqlcmd to work on Ubuntu 18.04\nRUN apt-get update && apt-get install -y libssl1.1 libssl-dev\n# in